In [2]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('val.csv')

In [4]:
train = train.sample(frac=1)

In [5]:
test = test.sample(frac=1)

In [6]:
train[train == 'na'] = np.nan

In [7]:
test[test == "na"] = np.nan

In [8]:
train = train.drop(['rowc', 'colc', 'ra', 'dec'], axis=1)

In [9]:
test = test.drop(['rowc', 'colc', 'ra', 'dec'], axis=1)

In [10]:
data = train.iloc[:, 1:39]

In [11]:
y = train.iloc[:, 39]

In [12]:
def norm_to_gauss(df):
    return (df - df.mean()) * 1.0 / (df.var())

In [13]:
data = data.apply(np.float64)

In [14]:
data = data.apply(norm_to_gauss)

In [15]:
data_test = test.iloc[:, 1:39]

In [16]:
data_test = data_test.apply(np.float64)

In [17]:
data_test = data_test.apply(norm_to_gauss)

In [110]:
from fancyimpute import MICE

In [175]:
data = MICE(n_imputations=10).complete(data)

In [ ]:
if data_test.isnull().any().any():
    data_test = MICE(n_imputations=10).complete(data_test)

In [20]:
def argmax(preds):
    return np.asarray([np.argmax(line) for line in preds])

In [21]:
from keras.utils.np_utils import to_categorical   

cl = to_categorical(y, num_classes=3)

Using TensorFlow backend.


In [22]:
from keras.layers.advanced_activations import *

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
model = Sequential()

model.add(Dense(256, init='uniform', input_dim=38, activation=PReLU()))
model.add(Dropout(0.5))
model.add(Dense(128, init='uniform', activation=PReLU()))
model.add(Dropout(0.3))
model.add(Dense(64, init='uniform', activation=PReLU()))
model.add(Dropout(0.2))
model.add(Dense(128, init='uniform', activation=PReLU()))
model.add(Dropout(0.2))
model.add(Dense(64, init='uniform', activation=PReLU()))
model.add(Dropout(0.2))
model.add(Dense(128, init='uniform', activation=PReLU()))
model.add(Dropout(0.2))
model.add(Dense(64, init='uniform', activation=PReLU()))
model.add(Dropout(0.2))
model.add(Dense(9, init='uniform', activation=PReLU()))
model.add(Dense(3, init='uniform', activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
hist = model.fit(data, cl, epochs=15, batch_size=500)

predictions = argmax(model.predict(data_test))


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, input_dim=38, kernel_initializer="uniform", activation=<keras.lay...)`
  """
/usr/local/lib/python3.5/dist-packages/keras/activations.py:115: UserWarning: Do not pass a layer instance (such as PReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_initializer="uniform", activation=<keras.lay...)`
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, kernel_initializer="uniform", activation=<keras.lay...)`
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11:

Epoch 1/15
30000/30000 [==============================] - 13s 431us/step - loss: 0.6083 - acc: 0.6806
Epoch 2/15
30000/30000 [==============================] - 1s 30us/step - loss: 0.5056 - acc: 0.7472
Epoch 3/15
30000/30000 [==============================] - 1s 29us/step - loss: 0.4033 - acc: 0.8304
Epoch 4/15
30000/30000 [==============================] - 1s 29us/step - loss: 0.3741 - acc: 0.8518
Epoch 5/15
30000/30000 [==============================] - 1s 29us/step - loss: 0.3638 - acc: 0.8568
Epoch 6/15
30000/30000 [==============================] - 1s 30us/step - loss: 0.3535 - acc: 0.8624
Epoch 7/15
30000/30000 [==============================] - 1s 29us/step - loss: 0.3493 - acc: 0.8632
Epoch 8/15
30000/30000 [==============================] - 1s 29us/step - loss: 0.3419 - acc: 0.8663
Epoch 9/15
30000/30000 [==============================] - 1s 29us/step - loss: 0.3373 - acc: 0.8679
Epoch 10/15
30000/30000 [==============================] - 1s 30us/step - loss: 0.3354 - acc: 0.86

In [25]:
from sklearn.metrics import f1_score
f1_score(true, predictions, average="macro")

0.7540964799570918

In [ ]:
model.summary()

In [24]:
true = np.asarray(test.iloc[:, 39])

In [27]:

df = pd.DataFrame(data={'objid': test.iloc[:, 0], 'prediction': predictions})

In [29]:
df.to_csv('predictions.csv', encoding='utf-8', index=False)